# nmap常用命令

### nmap简介

NMap，也就是Network Mapper，是Linux下的网络扫描和嗅探工具包。

其基本功能有三个：

1. 扫描主机端口，嗅探所提供的网络服务
2. 探测一组主机是否在线
3. 推断主机所用的操作系统，到达主机经过的路由，系统已开放端口的软件版本

### nmap端口状态解析

open ： 应用程序在该端口接收 TCP 连接或者 UDP 报文。 

closed ：关闭的端口对于nmap也是可访问的， 它接收nmap探测报文并作出响应。但没有应用程序在其上监听。

filtered ：由于包过滤阻止探测报文到达端口，nmap无法确定该端口是否开放。过滤可能来自专业的防火墙设备，路由规则 或者主机上的软件防火墙。

unfiltered ：未被过滤状态意味着端口可访问，但是nmap无法确定它是开放还是关闭。 只有用于映射防火墙规则集的 ACK 扫描才会把端口分类到这个状态。
open | filtered ：无法确定端口是开放还是被过滤， 开放的端口不响应就是一个例子。没有响应也可能意味着报文过滤器丢弃了探测报文或者它引发的任何反应。UDP，IP协议,FIN, Null 等扫描会引起。

closed|filtered：（关闭或者被过滤的）：无法确定端口是关闭的还是被过滤的

### 探测主机存活常用方式

-sP ：进行ping扫描

打印出对ping扫描做出响应的主机,不做进一步测试(如端口扫描或者操作系统探测)

-sn:  Ping Scan - disable port scan  #ping探测扫描主机， 不进行端口扫描 （测试过对方主机把icmp包都丢弃掉，依然能检测到对方开机状态）

-sA
nmap 10.0.1.161 -sA （发送tcp的ack包进行探测，可以探测主机是否存活）

### 端口扫描的高级用法　

 -sS ：半开放扫描（非3次握手的tcp扫描）
 
 使用频率最高的扫描选项：SYN扫描,又称为半开放扫描，它不打开一个完全的TCP连接，执行得很快，效率高
（一个完整的tcp连接需要3次握手，而-sS选项不需要3次握手）
Tcp SYN Scan (sS) 它被称为半开放扫描
优点：Nmap发送SYN包到远程主机，但是它不会产生任何会话，目标主机几乎不会把连接记入系统日志。（防止对方判断为扫描攻击），扫描速度快，效率高，在工作中使用频率最高
缺点：它需要root/administrator权限执行


sT：3次握手方式tcp的扫描

Tcp connect() scan (sT)和上面的Tcp SYN 对应，TCP connect()扫描就是默认的扫描模式.
不同于Tcp SYN扫描,Tcp connect()扫描需要完成三次握手,并且要求调用系统的connect().
优点：你勿需root权限。普通用户也可以使用。
缺点：这种扫描很容易被检测到，在目标主机的日志中会记录大批的连接请求以及错误信息，由于它要完成3次握手，效率低，速度慢，建议使用-sS 

sU：udp端口的扫描

Udp scan(sU) 顾名思义,这种扫描技术用来寻找目标主机打开的UDP端口.它不需要发送任何的SYN包，因为这种技术是针对UDP端口的。UDP扫描发送UDP数据包到目标主机，并等待响应,
如果返回ICMP不可达的错误消息，说明端口是关闭的，如果得到正确的适当的回应，说明端口是开放的.udp端口扫描速度比较慢

sF：也是tcp的扫描一种，发送一个FIN标志的数据包

FIN scan(sF)
有时候TcpSYN扫描不是最佳的扫描模式,因为有防火墙的存在.目标主机有时候可能有IDS和IPS系统的存在,防火墙会阻止掉SYN数据包。发送一个设置了FIN标志的数据包并不需要完成TCP的握手.
和sS扫描效果差不多，比sT速度快

-sF、-sX、-sN

秘密FIN数据包扫描、圣诞树(XmasTree)、空(Null)扫描模式
有的防火墙可能专门阻止-sS扫描。使用这些扫描可以发送特殊标记位的数据包
比如，-sF发送一个设置了FIN标志的数据包
它们和-sS一样也需要完成TCP的握手.
和sS扫描效果差不多，都比sT速度快
除了探测报文的标志位不同，三种扫描在行为上一致
优势：能躲过一些无状态防火墙和报文过滤路由器，比SYN还要隐秘
劣势：现代的IDS产品可以发现，并非所有的系统严格遵循RFC 793
 
 
即使SYN扫描都无法确定的情况下使用：一些防火墙和包过滤软件能够对发送到被限制端口的SYN数据包进行监视，
而且有些程序比如synlogger和courtney能够检测那些扫描。使用-sF、-sX、-sN可以逃过这些干扰。
这些扫描方式的理论依据是：关闭的端口需要对你的探测包回应RST包，而打开的端口必需忽略有问题的包。
FIN扫描使用暴露的FIN数据包来探测，而圣诞树扫描打开数据包的FIN、URG和PUSH标志。
由于微软决定完全忽略这个标准，另起炉灶。所以这种扫描方式对Windows无效。
不过，从另外的角度讲，可以使用这种方式来分别两种不同的平台。
如果使用这种扫描方式可以发现打开的端口，你就可以确定目标注意运行的不是Windows系统。
如果使用-sF、-sX或者-sN扫描显示所有的端口都是关闭的，而使用-sS（SYN）扫描显示有打开的端口，你可以确定目标主机可能运行的是Windwos系统。
现在这种方式没有什么太大的用处，因为nmap有内嵌的操作系统检测功能。还有其它几个系统使用和windows同样的处理方式，包括Cisco、BSDI、HP/UX、MYS、IRIX。
在应该抛弃数据包时，以上这些系统都会从打开的端口发出复位数据包。

sW：窗口扫描
Window扫描，即窗口扫描
当然也可以利用Window扫描方式，得出一些端口信息，可以与之前扫描分析的结果相互补充。Window扫描方式只对某些TCPIP协议栈才有效。
它也是基于tcp的扫描，个人感觉用处不大
另外我尝试使用它对某机器的22端口扫描，发现对方22端口状态居然是错误的。

sV：版本检测(sV)
版本检测是用来扫描目标主机和端口上运行的软件的版本，如下扫描，多出了ssh的版本信息 

### 不常用的命令

`--top-ports <number>: Scan <number> most common ports`
#扫描常用的端口，number如果写成10，那就是扫描最常用的10个端口。比如，ssh，http，ftp等热门端口
    
`--port-ratio <ratio>: Scan ports more common than <ratio>` #扫描常用端口里，占的比重在0.x 之上的端口
比如ratio=0.2 那么就是常用端口中占的分量超过0.2的端口，比如http的80端口

-sO：探测对方，TCP/IP协议簇中有哪些协议，类型号分别是多少

--allports
--allports (不为版本探测排除任何端口)经过我的测试，发现对于一些大的端口号，它没能检测出来 默认情况下,Nmap版本探测会跳过9100 TCP端口,因为一些打印机简单地打印送到该端口的任何数据,这回导致数十页HTTP get请求,二进制SSL会话请求等等被打印出来.这一行为可以通过修改或删除nmap-service-probes中的Exclude指示符改变,您也可以不理会任何Exclude指示符,指定--allports扫描所有端口

-P0
在扫描之前，不必ping主机。有些网络的防火墙不允许ICMPecho请求穿过，使用这个选项可以对这些网络进行扫描。microsoft.com就是一个例子，因此在扫描这个站点时，你应该一直使用-P0或者-PT80选项。

-PT
扫描之前，使用TCPping确定哪些主机正在运行。nmap不是通过发送ICMPecho请求包然后等待响应来实现这种功能，而是向目标网络(或者单一主机)发出TCPACK包然后等待回应。

-A参数，综合扫描。比较慢，不推荐
-A: Enable OS detection, version detection, script scanning, and traceroute #-A综合扫描，包括系统探测，版本探测，脚本扫描，路由跟踪

-e：指定网络接口,扫描时使用哪个网卡

用法 ：nmap -e eth0 targetip

-S：可以伪装源地址进行扫描。这样好处在于不会被对方发现自己的真实IP
把自己源地址伪装成10.0.1.167扫描A机器
nmap -e eth0 10.0.1.161 -S 10.0.1.167 -Pn

nmap -iflist：查看本地路由与接口
Nmap中提供了–iflist选项来查看本地主机的接口信息与路由信息。当遇到无法达到目标主机或想选择从多块网卡中某一特定网卡访问目标主机时，可以查看nmap –iflist中提供的网络接口信息。
和route  -n功能一样

### 小结：

nmap工具很强大。建议先用Nmap扫描一个熟悉的系统，感觉一下Nmap的基本运行模式，熟悉之后，再将扫描范围扩大到其他系统。<br>首先扫描内部网络看看Nmap报告的结果，然后从一个外部IP地址扫描，注意防火墙、入侵检测系统（IDS）以及其他工具对扫描操作的反应。
通常，TCP connect()会引起IDS系统的反应（默认的nmap扫描就是sT的方式，也就是3次握手的方式扫描）
但IDS不一定会记录俗称“半连接”的TCP SYN扫描（-sS方式的）。
 
如果你打算熟悉和使用Nmap，下面几点经验可能对你有帮助：
1、 避免误解。不要随意选择测试Nmap的扫描目标。许多单位把端口扫描视为恶意行为，所以测试Nmap最好在内部网络进行。如有必要，应该告诉同事你正在试验端口扫描，因为扫描可能引发IDS警报以及其他网络问题。
2、 关闭不必要的服务。根据Nmap提供的报告（同时考虑网络的安全要求），关闭不必要的服务，或者调整路由器的访问控制规则（ACL），禁用网络开放给外界的某些端口。
3、 建立安全基准。在Nmap的帮助下加固网络、搞清楚哪些系统和服务可能受到攻击之后，下一步是从这些已知的系统和服务出发建立一个安全基准，以后如果要启用新的服务或者服务器，就可以方便地根据这个安全基准执行。